# Lung Segmentation

## Prepare environement

In [1]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 7.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Libraries

In [ ]:
import os
import sys
import inspect
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt

import SimpleITK as sitk

from google.colab.patches import cv2_imshow

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

from scipy.ndimage import center_of_mass
from scipy.ndimage import binary_fill_holes, binary_closing, binary_dilation, binary_opening, binary_erosion
from skimage import measure, morphology



In [ ]:
path = "/content/drive/Shareddrives/IA DL_project/ML IA/IMAGE_PROCESSING_PIPELINE"

if path not in sys.path:
  sys.path.append(path)

import luna_module
from luna_module import *

# List all function names in the luna_module
function_names = [name for name, obj in inspect.getmembers(luna_module) if inspect.isfunction(obj)]
print(function_names)

['annotations_by_uid', 'binarize_lung', 'binarize_lung_3d', 'binary_closing', 'binary_dilation', 'binary_erosion', 'binary_fill_holes', 'binary_opening', 'center_of_mass', 'clear_border', 'convert_annotation_df', 'convert_annotation_df_with_uid', 'create_3d_mask', 'create_annotations_mask', 'create_annotations_mask_by_uid', 'create_patch', 'debugger', 'draw_ellipsoid', 'find_by_uid', 'find_neighborhood_indices', 'find_neighborhood_indices_more_precise', 'get_slice_candidates', 'get_slice_candidates_old', 'get_slices', 'get_uids', 'img_by_uid', 'masked_annotations_by_uid', 'masked_annotations_with_info_by_uid', 'meta_by_uid', 'norm2float', 'norm2uint16', 'norm2uint8', 'normalize_intensity', 'plot_slices', 'process_slice_candidates', 'process_slices', 'remove_non_central_objects', 'sensitivity_score', 'sensitivity_score_more_precise', 'show_3_images', 'subset_by_uid', 'unwanted_object_filter']


### Functions

In [ ]:
def imshow(img):
  return cv2_imshow(img)

def debugger(img, title=None):
    """
    Displays an image with its unique intensity values and an optional title.

    Args:
    img (numpy.ndarray): The image to be displayed, represented as a 2D array.
    title (str, optional): The title to be printed above the image. Default is None.

    Returns:
    None
    """
    if title is not None:
        print(title)
    print(np.unique(img))
    plt.imshow(img, cmap="gray")
    plt.show()

### Data paths

In [ ]:
LUNA_PATH = os.path.join(os.getcwd(), "drive", "Shareddrives", "IA DL_project", "ML IA", "LUNA16")

SUBSETS_PATH = os.path.join(LUNA_PATH, "subsets") # path for subsets folder
SUBSETS = os.listdir(SUBSETS_PATH) # subset folders present
SUBSET = "subset0"

ANNOTATIONS_DF = pd.read_csv(os.path.join(LUNA_PATH, f"{SUBSET}_annotations_expanded.csv"), index_col="Unnamed: 0")

______________
## Image loading

In [ ]:
image_dict = {}

In [ ]:
read_in_num = 40

subset_path = lambda subset: os.path.join(SUBSETS_PATH, subset)
file_path = lambda subset, filename: os.path.join(SUBSETS_PATH, subset, filename)

for i, subset in enumerate(SUBSETS):
  if subset != SUBSET:
    break

  # Create subset key if it doesn't exist
  if subset not in image_dict:
    image_dict[subset] = {}

  # Iterate through all the keys in the subset
  read_in = 0
  for j, filename in enumerate(os.listdir(subset_path(subset))):
    # if j < 8: continue # Do not read in before the third file (they have annotations)

    name, extension = os.path.splitext(filename) # Separate filename and extension

    if extension != ".mhd":
      continue

    if not len(annotations_by_uid(name, ANNOTATIONS_DF)):
      print(f"[SKIP] - no annotations for {name}")
      continue

    # Only read if there is missing information
    if name not in image_dict[subset]:
      print(f"[START] -- READING IN: {subset}/{name}")
      image = sitk.ReadImage(os.path.join(SUBSETS_PATH, subset, filename))

      image_dict[subset][name] = image
      print(f"[DONE] -- READING IN: {subset}/{name}")

    else:
      print(f"[SKIP] -- ALREADY EXISTS: {subset}/{name}")
      continue

    read_in += 1 # Increment file-read-in counter
    if read_in >= read_in_num: break # If desired read-ins are reached for loop is broken in the subset

  print(f"[STATUS] -- READ IN {read_in_num} entries in {subset}")


subset_keys = list(image_dict.keys()) # Extract the stored subset-keys
filename_keys = [list(image_dict[key].keys()) for key in subset_keys] # Extract the stored filename-keys
filename_keys_flat = list(np.array(filename_keys).ravel())#[*filename_keys[0], *filename_keys[1],] # Flatten the stored filename-keys

print(f"\n")
print(f"Subset keys: {subset_keys}")
print(f"Subset lengths: {[len(image_dict[key]) for key in subset_keys]}")
print(f"Filename keys: {filename_keys_flat}")

[SKIP] - no annotations for 1.3.6.1.4.1.14519.5.2.1.6279.6001.105756658031515062000744821260
[START] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492
[DONE] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492
[START] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059
[DONE] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059
[START] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987
[DONE] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987
[SKIP] - no annotations for 1.3.6.1.4.1.14519.5.2.1.6279.6001.122763913896761494371822656720
[START] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565
[DONE] -- READING IN: subset0/1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565
[SKIP] - no annotations for 1.3.6.1.4.1.14519.5.2.

_______________________
## MASK

In [ ]:
def draw_ellipsoid(mask, center, diameters, zyx=True):
    """
    Draws ellipsioid with custom diameters on a 3D image
    NOTE: center coordinates are in (z,y,x) order by default
    """
    # Extract the diameters for each axis
    if zyx:
      dz, dy, dx = diameters
    else:
      dx, dy, dz = diameters

    # Calculate the radii for each axis
    rz, ry, rx = dz / 2, dy / 2, dx / 2

    # Extract the shape of the array
    zmax, ymax, xmax = mask.shape

    # Get the range of indices to iterate over (stay within the array bounds)
    z_start, z_end = max(0, center[0]-int(rz)), min(zmax, center[0]+int(rz)+1)
    y_start, y_end = max(0, center[1]-int(ry)), min(ymax, center[1]+int(ry)+1)
    x_start, x_end = max(0, center[2]-int(rx)), min(xmax, center[2]+int(rx)+1)

    # Iterate over each point in the bounding box of the ellipsoid
    for z in range(z_start, z_end):
        for y in range(y_start, y_end):
            for x in range(x_start, x_end):
                # Calculate the normalized distance from the current point to the center
                if (((x - center[2])**2 / rx**2) + ((y - center[1])**2 / ry**2) + ((z - center[0])**2 / rz**2)) <= 1:
                    mask[z, y, x] = 1  # Set the value inside the ellipsoid


def create_3d_mask(center, diameters, img=np.zeros((28,28,28))):
  mask = np.zeros_like(img)
  draw_ellipsoid(mask, center, diameters)
  return mask

def create_annotations_mask(origin, spacing, annotations:pd.DataFrame, image:np.ndarray, verbose=False):
  """
  Creates 3D mask for annotations and provided metadata and CT scan case
  """
  centers = []
  diameters = []

  # If there is no given annotations
  if not len(annotations):
    return np.zeros_like(image, dtype=np.uint8), []


  # Iterate thorugh the annotations
  annotation_i = 0 # Toggle mask creation in the first iteration
  for i, row in annotations.iterrows():
    world_coords = np.array([row['coordX'], row['coordY'], row['coordZ']]) # Coordinates of the device
    pixel_coords = np.round((world_coords - origin) / spacing).astype(int) # Translated coordinates into voxelspace

    diam = row["diameter_mm"] # Diameter scalar
    diams = np.rint(diam / np.array(spacing)) # Diameters scaled by spacing (x, y, z)

    # Flipping axes: (x, y, z) -> (z, y, x)
    v_center = np.array([pixel_coords[2], pixel_coords[1], pixel_coords[0],]) # nodule center in voxel space (z,y,x ordering)
    v_diameters = np.array([diams[2], diams[1], diams[0],]) # flipping to (z, y, x)

    centers.append(v_center)
    diameters.append(v_diameters)


    # Adding annotations to the mask
    if annotation_i == 0:
      if verbose:
        print(f"[ANNOTATION #{annotation_i + 1}] -- Create image mask")

      mask = create_3d_mask(v_center, v_diameters, image)
      annotation_i += 1
    else:
      if verbose:
        print(f"[ANNOTATION #{annotation_i + 1}] -- Adding annotation #{annotation_i + 1}, index: {i}")

      draw_ellipsoid(mask, v_center, v_diameters)
      annotation_i += 1

    if verbose:
      z, y, x = image.shape
      print(f"Row index: {i}")
      print(f"Slices: Z: {z}, Y: {y}, X: {x}")
      print(f"Origin : {origin}")
      print(f"Spacing : {spacing}")
      print(f"V-center: {v_center}")
      print(f"Diameter: {diam}")
      print(f"V-diameters: {v_diameters}")

  return mask, np.array(centers)


def create_annotations_mask_by_id(uid, verbose=False):
  origin, spacing = meta_by_uid(uid)
  annotations = annotations_by_uid(uid)
  img = img_by_uid(uid)

  return create_annotations_mask(origin, spacing, annotations, img, verbose)

______
## ANNOTATIONS

In [ ]:
current_files = [key for keys in filename_keys for key in keys]
filtered_df = ANNOTATIONS_DF[ANNOTATIONS_DF["seriesuid"].isin(current_files)]

In [ ]:
def create_annotations_mask(origin, spacing, annotations:pd.DataFrame, image:np.ndarray, verbose=False):
  """
  Creates 3D mask for annotations and provided metadata and CT scan case
  """
  centers = []
  diameters = []

  # If there is no given annotations
  if not len(annotations):
    return np.zeros_like(image, dtype=np.uint16), []


  # Iterate thorugh the annotations
  annotation_i = 0 # Toggle mask creation in the first iteration
  for i, row in annotations.iterrows():
    world_coords = np.array([row['coordX'], row['coordY'], row['coordZ']]) # Coordinates of the device
    pixel_coords = np.round((world_coords - origin) / spacing).astype(int) # Translated coordinates into voxelspace

    diam = row["diameter_mm"] # Diameter scalar
    diams = np.rint(diam / np.array(spacing)) # Diameters scaled by spacing (x, y, z)

    # Flipping axes: (x, y, z) -> (z, y, x)
    v_center = np.array([pixel_coords[2], pixel_coords[1], pixel_coords[0],]) # nodule center in voxel space (z,y,x ordering)
    v_diameters = np.array([diams[2], diams[1], diams[0],]) # flipping to (z, y, x)

    centers.append(v_center)
    diameters.append(v_diameters)


    # Adding annotations to the mask
    if annotation_i == 0:
      if verbose:
        print(f"[ANNOTATION #{annotation_i + 1}] -- Create image mask")

      mask = create_3d_mask(v_center, v_diameters, image)
      annotation_i += 1
    else:
      if verbose:
        print(f"[ANNOTATION #{annotation_i + 1}] -- Adding annotation #{annotation_i + 1}, index: {i}")

      draw_ellipsoid(mask, v_center, v_diameters)
      annotation_i += 1

    if verbose:
      z, y, x = image.shape
      print(f"Row index: {i}")
      print(f"Slices: Z: {z}, Y: {y}, X: {x}")
      print(f"Origin : {origin}")
      print(f"Spacing : {spacing}")
      print(f"V-center: {v_center}")
      print(f"Diameter: {diam}")
      print(f"V-diameters: {v_diameters}")

  return mask, np.array(centers)


def create_annotations_mask_by_uid(uid, image_dict, verbose=False):
  origin, spacing = meta_by_uid(uid, image_dict)
  annotations = annotations_by_uid(uid, ANNOTATIONS_DF)
  img = img_by_uid(uid, image_dict)

  return create_annotations_mask(origin, spacing, annotations, img, verbose)

________________

## LUNG

TEST ON ONE Z SLICE

In [ ]:
uids = get_uids(image_dict)
uids = pd.Series(np.zeros((len(uids)), int), index=uids)
# print("ALL")
# print(uids)

ann = filtered_df["seriesuid"].value_counts()

# print("W/ ANNOTATIONS")
# print(filtered_df["seriesuid"].value_counts())

In [ ]:
# Combine s1 with s2
all = ann.combine_first(uids).astype(int)

print("Annotations for each case")
print(all)
#print(find_by_uid("1.3.6.1.4.1.14519.5.2.1.6279.6001.219909753224298157409438012179"))

Annotations for each case
1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059    2
1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048929153767743874565    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.126264578931778258890371755354    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.128023902651233986592378348912    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.129055977637338639741695800950    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.130438550890816550994739120843    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.134996872583497382954024478441    4
1.3.6.1.4.1.14519.5.2.1.6279.6001.137763212752154081977261297097    2
1.3.6.1.4.1.14519.5.2.1.6279.6001.138080888843357047811238713686    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.139258777898746693365877042411    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.141069661700670042960678408762    1
1.3.6.1.4.1.14519.5.2.1.6279.6001.154677396354641150280013275227

In [ ]:
def delete_table(slc, debug=False):
    new_slc = slc.copy()
    labels = measure.label(slc, background=0)
    idxs = np.unique(labels)[1:]
    COM_ys = np.array([center_of_mass(labels==i)[0] for i in idxs])

    masses = [center_of_mass(labels == j) for j in range(1, np.unique(labels).size)]

    if debug:
      plt.imshow(new_slc)
      plt.show()
      plt.imshow(labels)
      plt.show()
      print(masses)

    # Calculate distances from the center of the slice
    center_y = slc.shape[0] / 2
    distances = [(i, np.abs(com[0] - center_y)) for i, com in zip(idxs, masses)]

    # Sort by distance and keep the closest 3 labels
    distances.sort(key=lambda x: x[1])
    closest_idxs = [distances[i][0] for i in range(min(3, len(distances)))]

    for idx in idxs:
        if idx not in closest_idxs:
            new_slc[labels == idx] = 0

    for idx, COM_y in zip(idxs, COM_ys):
        if (COM_y < 0.25*slc.shape[0]):
            new_slc[labels==idx] = 0
        elif (COM_y > 0.75*slc.shape[0]):
            new_slc[labels==idx] = 0

    return new_slc

def unwanted_object_filter(img: np.ndarray) -> np.ndarray:
  closed = binary_closing(img, morphology.disk((5)))

  # Label connected components
  labels = measure.label(closed)
  properties = measure.regionprops(labels)

  # Sort regions by area and keep the two largest
  detected_objects = sorted(properties, key=lambda x: x.area, reverse=True)

  # Filter out the relevant sizes
  area_th = 120
  detected_objects = [obj for obj in detected_objects if obj.area > area_th]

  # Create an empty image to hold the result
  filtered_image = np.zeros_like(img, dtype=bool)

  # Fill in the regions of the two largest objects
  for prop in detected_objects:
      filtered_image[labels == prop.label] = True

  filtered_image = delete_table(filtered_image)

  return filtered_image

In [ ]:
air_th = -1000
lung_th = -500
def lung_mask(img, denoising="gauss", dilation_kern=3, debug=False):
  analyzed = img.copy()

  # Clipping uninterpretable data
  analyzed[analyzed < air_th] = air_th

  kernel = 5
  match denoising:
    case "median":
      analyzed = cv.medianBlur(analyzed, ksize=kernel)
    case "gauss":
      analyzed = cv.GaussianBlur(analyzed, (kernel, kernel), sigmaX=1.5)
    case "none":
      analyzed = analyzed

  # Filtering out tissues
  thresholded = analyzed < lung_th
  if debug:
    plt.imshow(thresholded)
    plt.show()

  # Inverting to get the body area
  inverted = ~thresholded
  if debug:
    plt.imshow(inverted)
    plt.show()

  # Filling body area
  filled_inverted = binary_fill_holes(inverted)
  if debug:
    plt.imshow(filled_inverted)
    plt.show()

  # Eroding in order to remove body periphery artifacts
  filled_inverted = binary_erosion(filled_inverted, morphology.disk(13))
  if debug:
    plt.imshow(filled_inverted)
    plt.show()

  # AND the body area
  lung = np.bitwise_and(filled_inverted, thresholded)
  if debug:
    plt.imshow(filled_inverted)
    plt.show()

  # Removing non-lung objects
  lung = unwanted_object_filter(lung)
  if debug:
    plt.imshow(lung)
    plt.show()

  # Fill holes in lung
  lung = binary_closing(lung, morphology.disk(15))
  if debug:
    plt.imshow(lung)
    plt.show()

  # Dilation
  if dilation_kern:
    lung = cv.dilate(lung.astype(float), cv.getStructuringElement(cv.MORPH_ELLIPSE, (dilation_kern, dilation_kern)))
  if debug:
    plt.imshow(lung)
    plt.show()

  return lung.astype(bool)

def lung_mask_3d(img, denoising="gauss", dilation_kern=5):
  if len(img.shape) != 3:
    raise Exception("Shape has to be 3 dimensional")
  return process_slices(img, 0, lung_mask)

def remove_non_central_objects(img, debug=False):
    """
    Deletes objects from the input binary image slice that are not within the central region.

    Parameters:
    img (ndarray): Input binary image slice.
    debug (bool): If True, shows debugging information (default is False).

    Returns:
    filtered_img (ndarray): Modified binary image slice with certain objects removed.
    """
    # Create a copy of the input slice to
    filtered_img = img.copy()

    # Label connected components in the slice
    labels = measure.label(img, background=0)
    idxs = np.unique(labels)[1:]  # Get unique labels, excluding the background

    # Compute the center of mass for each label
    COM_ys = np.array([center_of_mass(labels == i)[0] for i in idxs])
    masses = [center_of_mass(labels == j)
              for j in range(1, np.unique(labels).size)]

    if debug:
        # Display debugging information if debug is True
        plt.imshow(filtered_img)
        plt.show()
        plt.imshow(labels)
        plt.show()
        print(masses)

    # Calculate distances from the center of the slice
    center_y = img.shape[0] / 2
    distances = [(i, np.abs(com[0] - center_y))
                 for i, com in zip(idxs, masses)]

    # Sort by distance and keep the closest 3 labels
    distances.sort(key=lambda x: x[1])
    closest_idxs = [distances[i][0] for i in range(min(3, len(distances)))]

    # Remove labels that are not among the closest 3
    for idx in idxs:
        if idx not in closest_idxs:
            filtered_img[labels == idx] = 0

    # Further filter labels based on their vertical position in the slice
    for idx, COM_y in zip(idxs, COM_ys):
        if COM_y < 0.25 * img.shape[0] or COM_y > 0.75 * img.shape[0]:
            filtered_img[labels == idx] = 0

    return filtered_img


def unwanted_object_filter(img: np.ndarray, area_th=1500) -> np.ndarray:
    """
    Filters out unwanted objects from a binary image based on area and location.

    Parameters:
    img (ndarray): Input binary image.

    Returns:
    filtered_image (ndarray): Binary image with unwanted objects removed.
    """
    # Perform binary closing to close small gaps in the image
    closed = binary_closing(img.copy(), morphology.disk(5))

    # Label connected components in the closed image
    labels = measure.label(closed)
    properties = measure.regionprops(labels)

    # Sort regions by area and keep those larger than a threshold
    detected_objects = [obj for obj in properties if obj.area > area_th]

    # Create an empty image to hold the result
    filtered_image = np.zeros_like(img, dtype=bool)

    # Fill in the regions of the relevant objects
    for prop in detected_objects:
        filtered_image[labels == prop.label] = True

    # Remove tables or similar objects using the delete_table function
    filtered_image = remove_non_central_objects(filtered_image)

    return filtered_image


def binarize_lung(img: np.ndarray, threshold_val=LUNG_TH, debug=False):
    """
    Binarizes a lung image using a series of image processing steps.

    This function applies Gaussian blur, thresholding, and several morphological operations
    to binarize the lung image, removing unwanted artifacts and filling holes.

    Args:
    img (numpy.ndarray): The input lung image.
    threshold_val (int or float): The threshold value for binarization.

    Returns:
    numpy.ndarray: The binarized lung image.
    """
    # Apply Gaussian blur to the image
    blurred = cv.GaussianBlur(img, (5, 5), 0)

    if debug:
      debugger(norm2uint16(blurred), "blurred")

    # Apply thresholding to create a binary image
    th = (blurred < threshold_val).astype(float)

    if debug:
      debugger((th), "thresholded")


    # Remove objects connected to the image border
    th = clear_border(th)

    if debug:
      debugger((th), "clear_border")

    # Remove small unwanted objects
    th = unwanted_object_filter(th)

    if debug:
      debugger((th), "object_filter")

    # Apply binary closing to smooth edges and remove small holes
    th = binary_closing(th, morphology.disk(3))

    if debug:
      debugger((th), "closed")

    # Fill remaining larger holes
    th = binary_fill_holes(th)

    if debug:
      debugger((th), "fill holes")


    return th


def binarize_lung_3d(img_3d, threshold_val=LUNG_TH):
    """
    Binarizes a 3D lung image using a specified threshold value.

    Args:
    img_3d (numpy.ndarray): The input 3D lung image.
    threshold_val (int): The threshold value for binarization.

    Returns:
    numpy.ndarray: The binarized 3D lung image.

    Raises:
    Exception: If the input image is not 3-dimensional.
    """
    if len(img_3d.shape) != 3:
        raise Exception("Shape has to be 3 dimensional")
    return process_slices(img_3d, 0, binarize_lung)

def debugger(img, title=None):
    """
    Displays an image with its unique intensity values and an optional title.

    Args:
    img (numpy.ndarray): The image to be displayed, represented as a 2D array.
    title (str, optional): The title to be printed above the image. Default is None.

    Returns:
    None
    """
    if title is not None:
        print(title)
    print(np.unique(img))
    plt.imshow(img, cmap="gray")
    plt.axis('off')  # Turn off axis ticks and labels
    plt.show()

In [ ]:
for i in range(len(filename_keys_flat)):
  img = img_by_uid(filename_keys_flat[i], image_dict)
  print(f"i: {i}")
  print(f"uid: {filename_keys_flat[i]}")
  mask, centers = create_annotations_mask_by_uid(filename_keys_flat[i], image_dict)
  print(centers)
  slice_i = centers[0][0]

  im = img_by_uid(filename_keys_flat[i], image_dict)[slice_i,:,:]
  im[im<AIR_TH] = AIR_TH
  debugger(im)


  lung_mask = binarize_lung(im, debug=True)

  masked = im.copy()
  masked[lung_mask==False] = AIR_TH

  debugger(masked)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
i = 26
img = img_by_uid(filename_keys_flat[i], image_dict)
filename_keys_flat[i]

In [ ]:
mask, centers = create_annotations_mask_by_uid(filename_keys_flat[i], image_dict)
print(centers)

slice_i = centers[0][0]

plt.imshow((img[slice_i,:,:]))
plt.show()
plt.imshow(binarize_lung(img_by_uid(filename_keys_flat[i], image_dict)[slice_i,:,:], debug=True))


In [ ]:
# plot_slices(get_slices(img, 0))
# plot_slices(get_slices(lung_mask_3d(img), 0))

In [ ]:
img.shape

In [ ]:
pairs = []

images = get_slices(img, 0)
lung_masks = get_slices(lung_mask_3d(img), 0)

for i in range(len(images)):
  pairs.append(images[i])
  pairs.append(lung_masks[i])

plot_slices(pairs)

__________

In [ ]:
annotation_masks = []
lung_masks = []
ims = []

for i, key in enumerate(filename_keys_flat):
  print("Centers")
  print(i)

  img = img_by_uid(key, image_dict)
  annotations = annotations_by_uid(key, ANNOTATIONS_DF)
  mask, centers = create_annotations_mask_by_uid(key, image_dict)
  print(centers)

  #print(np.unique(mask[centers[0][0], :, :]))

  slice_i = centers[0][0] if len(centers) else int(img.shape[0] / 2)
  ims.append(img[slice_i, :, :])


  l_mask = lung_mask(img[slice_i, :, :])
  segmented = l_mask * img[slice_i,:,:]
  annotation_masks.append(mask[slice_i, :, :])
  lung_masks.append(l_mask)

In [ ]:
# pairs = []

for i, key in enumerate(filename_keys_flat):
  im = ims[i]
  # pairs.append(im.copy())

  # pairs.append(lung_masks[i])

  im[~lung_masks[i]] = im.min()
  # pairs.append(im)

  show_3_images([im.copy(), lung_masks[i], im])
# plot_slices(pairs, titles=True, cols=3)